<a href="https://colab.research.google.com/github/joexu22/ESRGAN/blob/master/colab/ESRGAN_Inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use ESRGAN To Inpaint Borders

In [ ]:
#@title Check your GPU and your VRAM
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
%cd "/content/drive/MyDrive/008 - AI and Me/3_Resources"
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-' -O "RRDB_ESRGAN_x4.pth"
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr' -O "RRDB_PSNR_x4.pth"
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/x99pKzS7TNaErrC/download" -O "4x_fatal_Anime_500000_G.pth"
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/mDGmi7NgdyyQRXL/download?path=%2F&files=4x_FatalPixels_340000_G.pth&downloadStartSecret=r4q3aw60ijm" -O "4x_FatalPixels_340000_G.pth"
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download" -O "1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth"
!gdown --id 14lA-Ks5quxheNyVeXRvzeoSAOm6ISDHn -O "4x_NMKD-YandereNeo-Lite_320k.pth"
"""

In [ ]:
!pip install typer
!pip install rich

In [ ]:
%cd "/content"
!git clone https://github.com/joexu22/ESRGAN.git ESRGAN_Inpainting

# New Section

In [ ]:
from google.colab import files
from pathlib import Path
import PIL
from PIL import Image, ImageDraw
import shutil
import os

In [ ]:
# this adds either a cloumn or row to a image
def add_pixel_layer(image_width, image_height, image):
  if (image_height > image_width):
    canvas1 = PIL.Image.new('RGBA', (image_width+1, image_height), "#00ff00")
    canvas1.paste(image, (0,0)) # right column green
    if (image_height > image_width):
      canvas2 = PIL.Image.new('RGBA', (image_width+2, image_height), "#00ff00")
      canvas2.paste(canvas1, (1,0)) # left column green
      return canvas2
  elif (image_height < image_width):
    canvas1 = PIL.Image.new('RGBA', (image_width, image_height+1), "#00ff00")
    canvas1.paste(image, (0,1)) # top row green
    if (image_height < image_width):
      canvas2 = PIL.Image.new('RGBA', (image_width, image_height+2), "#00ff00")
      canvas2.paste(canvas1, (0,0)) # bottom row green
      return canvas2
  else:
    canvas = PIL.Image.new('RGBA', (image_width+1, image_height), "#00ff00")
    canvas.paste(image, (0,0))
    return canvas

In [ ]:
%cd "/content/ESRGAN_Inpainting"
!mkdir results

# initial image
for filename in [str(result) for result in list(Path('/content/ESRGAN_Inpainting/input').iterdir()) if str(result).endswith('.png')]:
  input_image = Image.open(filename)

# find target dimension
target_dimensions = max(input_image.size) # max of (width, height)
print("The Current Dimension We Have Is - {0}x{1}".format(input_image.size[0], input_image.size[1]))
print("The Target Dimension We Want Is - {0}x{0}".format(target_dimensions))

In [ ]:
# TODO function with pathname and optional name
# deal with compression artifacts?
!python upscale.py '/content/drive/MyDrive/008 - AI and Me/3_Resources/1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth' --input input/ --output output/
## move output to input
### delete input
shutil.rmtree('/content/ESRGAN_Inpainting/input', ignore_errors=True)
if not os.path.exists('/content/ESRGAN_Inpainting/input'):
  os.mkdir('/content/ESRGAN_Inpainting/input')
### move output to input
for filename in [str(output) for output in list(Path('/content/ESRGAN_Inpainting/output').iterdir()) if str(output).endswith('.png')]:
  output_image = Image.open(filename)
output_image.save('/content/ESRGAN_Inpainting/input/processed_img.png')
### delete output
shutil.rmtree('/content/ESRGAN_Inpainting/output', ignore_errors=True)
if not os.path.exists('/content/ESRGAN_Inpainting/output'):
  os.mkdir('/content/ESRGAN_Inpainting/output')

In [ ]:
%cd "/content/ESRGAN_Inpainting"
image_w = 0
image_h = 0
target_d = target_dimensions

input_path = '/content/ESRGAN_Inpainting/input/'
output_path = '/content/ESRGAN_Inpainting/output/'
results_path = '/content/ESRGAN_Inpainting/results/'

while image_w < target_d or image_h < target_d:
  # add layer
  for filename in [str(result) for result in list(Path(input_path).iterdir()) if str(result).endswith('.png')]:
    before_image = Image.open(filename)
  
  # keep before image in memory
  old_img = before_image

  # image has #00ff00 layer added
  prep_img = add_pixel_layer(before_image.size[0], before_image.size[1], before_image)

  # process the image
  ## save the 'prepared' image as new input
  shutil.rmtree(input_path, ignore_errors=True)
  if not os.path.exists(input_path):
    os.mkdir(input_path)
  prep_img.save(input_path + 'prepared_img.png')

  ## use the model
  !python upscale.py "/content/drive/MyDrive/008 - AI and Me/3_Resources/1x_NMKD-YandereInpaint_375000_G.pth" --input input/ --output output/

  # manage the ouput/processed image
  for filename in [str(output) for output in list(Path(output_path).iterdir()) if str(output).endswith('.png')]:
    output_image = Image.open(filename)
  
  ## paste original image back in
  canvas = PIL.Image.new('RGBA', (output_image.size[0], output_image.size[1]), 'white')
  canvas.paste(output_image, (0,0))
  canvas.paste(old_img, (1,0))

  ## save a copy to results
  result_image_name = "result-img-{0:05d}.png".format(canvas.size[0])
  canvas.save(results_path + '{0}'.format(result_image_name))

  ## save a copy to input
  ### first remove existing input
  shutil.rmtree(input_path, ignore_errors=True)
  if not os.path.exists(input_path):
    os.mkdir(input_path)
  ### then save to input
  canvas.save(input_path + 'processed_img.png')

  ## remove from output
  shutil.rmtree(output_path, ignore_errors=True)
  if not os.path.exists(output_path):
    os.mkdir(output_path)

  # increment iterator
  image_w = canvas.size[0]
  image_h = canvas.size[1]

In [ ]:
import imageio
import numpy as np

results_folder = "/content/ESRGAN_Inpainting/results"
target_d = target_dimensions

# horizontal only
def make_project_progress_mp4():
  result_images_list = [x for x in Path(results_folder).iterdir() if x.name.endswith('png') and 'result-img' in x.name]
  result_images_list.sort()
  # movie name
  movieName = results_folder + '/result_video.mp4'
  with imageio.get_writer(movieName, mode="I") as writer:
    for img_path in result_images_list:
      canvas = PIL.Image.new('RGBA', (target_d, target_d), 'white')
      img = Image.fromarray(imageio.imread(img_path))
      offset_center = (target_d-img.size[0])//2 # horizontal
      canvas.paste(img, (offset_center, 0))
      writer.append_data(np.array(canvas))

make_project_progress_mp4()


In [ ]:
import time

# move video to save folder
save_video_dir = '/content/drive/MyDrive/Random Drawings/InterpVideo/'
if not os.path.exists(save_video_dir):
  os.mkdir(save_video_dir)
save_video_path = save_video_dir + 'result_video_{0}.mp4'.format(int(time.time()))

shutil.copy('/content/ESRGAN_Inpainting/results/result_video.mp4', save_video_path)

In [ ]:
# this makes greenscreen
image_location = "/content/drive/MyDrive/008 - AI and Me/4_Inpainting/input/CowboyAngel.png"

def greenscreen(image_location):
  target_image = Image.open(Path(image_location))
  max_dimension = max(target_image.size[0], target_image.size[1])
  print(max_dimension)
  print(target_image.size[0])
  print(target_image.size[1])
  while target_image.size[0] < max_dimension or target_image.size[1] < max_dimension:
    print(target_image.size[0])
    print(target_image.size[1])
    target_image = add_pixel_layer(target_image.size[0], target_image.size[1], target_image)
  return target_image

# greenscreen(image_location)
